In [1]:
import re
import collections

In [2]:
string = 'low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new'
chars = [char for char in string]
Unique = list(set(chars))
l = [[0 for i in range(len(Unique))] for i in range(2)] 
l[0] = Unique
for i in range(len(Unique)):
    l[1][i]= chars.count(Unique[i])
l

[['n', 'i', 'o', ' ', 'd', 't', 'l', 'e', 'r', 'w', 's'],
 [8, 3, 7, 17, 3, 2, 7, 19, 9, 18, 2]]

In [3]:
## get the word frequency and add the end of word (</w>) token ## at the end of each word

patterns = "([A-Z]+\.)+|\$?\d+(\.\d+)?%?|\w+(-\w+)?(\.\w)?|\.+?"
Token = []
for i in re.finditer(patterns,string):
    Token.append(i.group())
tokens = list(set(Token))
print(tokens)
print(f"Vocabulary size: {len(tokens)}")

['newer', 'wider', 'low', 'lowest', 'new']
Vocabulary size: 5


In [4]:
word_freq_dict = collections.defaultdict(int)

for word in Token:
    word_freq_dict[' '.join(word) + ' _'] += 1

word_freq_dict

defaultdict(int,
            {'l o w _': 5,
             'l o w e s t _': 2,
             'n e w e r _': 6,
             'w i d e r _': 3,
             'n e w _': 2})

In [5]:
char_freq_dict = collections.defaultdict(int)
for word, freq in word_freq_dict.items():
    chars = word.split()
    for char in chars:
        char_freq_dict[char] += freq

char_freq_dict

defaultdict(int,
            {'l': 7,
             'o': 7,
             'w': 18,
             '_': 18,
             'e': 19,
             's': 2,
             't': 2,
             'n': 8,
             'r': 9,
             'i': 3,
             'd': 3})

In [6]:
import re

## create all possible consecutive pairs
pairs = collections.defaultdict(int)
for word, freq in word_freq_dict.items():
    chars = word.split()
    for i in range(len(chars)-1):
        pairs[chars[i], chars[i+1]] += freq

pairs

defaultdict(int,
            {('l', 'o'): 7,
             ('o', 'w'): 7,
             ('w', '_'): 7,
             ('w', 'e'): 8,
             ('e', 's'): 2,
             ('s', 't'): 2,
             ('t', '_'): 2,
             ('n', 'e'): 8,
             ('e', 'w'): 8,
             ('e', 'r'): 9,
             ('r', '_'): 9,
             ('w', 'i'): 3,
             ('i', 'd'): 3,
             ('d', 'e'): 3})

In [7]:
max(pairs, key=pairs.get)

('e', 'r')

In [8]:
##find the best pair

def get_pairs(word_freq_dict):
    pairs = collections.defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for i in range(len(chars)-1):
            pairs[chars[i], chars[i+1]] += freq
    return pairs

def merge_byte_pairs(best_pair, word_freq_dict):
    print(best_pair)
    merged_dict = {}
    bigram = re.escape(' '.join(best_pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in word_freq_dict:
        # print(word)
        w_out = p.sub(''.join(best_pair), word)
        merged_dict[w_out] = word_freq_dict[word]
    return merged_dict

def get_subword_tokens(word_freq_dict):
    char_freq_dict = collections.defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for char in chars:
            char_freq_dict[char] += freq
    return char_freq_dict

In [9]:
def BYTE_PAIR_ENCODING(iteration, word_freq_dict):
    Vocab = list(char_freq_dict.keys())
    try:
        for i in range(iteration):
            pairs = get_pairs(word_freq_dict)
            best_pair = max(pairs, key=pairs.get)
            print(f"Iteration {i}: ")
            word_freq_dict = merge_byte_pairs(best_pair, word_freq_dict)
            # print(word_freq_dict)
            subword_tokens = get_subword_tokens(word_freq_dict)
            print(subword_tokens)
            print(len(subword_tokens))
            print("--------")
            Vocab.append(''.join(best_pair))
    except:
        return Vocab

    return Vocab

In [10]:
Vocab = BYTE_PAIR_ENCODING(50, word_freq_dict)

Iteration 0: 
('e', 'r')
defaultdict(<class 'int'>, {'l': 7, 'o': 7, 'w': 18, '_': 18, 'e': 10, 's': 2, 't': 2, 'n': 8, 'er': 9, 'i': 3, 'd': 3})
11
--------
Iteration 1: 
('er', '_')
defaultdict(<class 'int'>, {'l': 7, 'o': 7, 'w': 18, '_': 9, 'e': 10, 's': 2, 't': 2, 'n': 8, 'er_': 9, 'i': 3, 'd': 3})
11
--------
Iteration 2: 
('n', 'e')
defaultdict(<class 'int'>, {'l': 7, 'o': 7, 'w': 18, '_': 9, 'e': 2, 's': 2, 't': 2, 'ne': 8, 'er_': 9, 'i': 3, 'd': 3})
11
--------
Iteration 3: 
('ne', 'w')
defaultdict(<class 'int'>, {'l': 7, 'o': 7, 'w': 10, '_': 9, 'e': 2, 's': 2, 't': 2, 'new': 8, 'er_': 9, 'i': 3, 'd': 3})
11
--------
Iteration 4: 
('l', 'o')
defaultdict(<class 'int'>, {'lo': 7, 'w': 10, '_': 9, 'e': 2, 's': 2, 't': 2, 'new': 8, 'er_': 9, 'i': 3, 'd': 3})
10
--------
Iteration 5: 
('lo', 'w')
defaultdict(<class 'int'>, {'low': 7, '_': 9, 'e': 2, 's': 2, 't': 2, 'new': 8, 'er_': 9, 'w': 3, 'i': 3, 'd': 3})
10
--------
Iteration 6: 
('new', 'er_')
defaultdict(<class 'int'>, {'lo

In [11]:
print('Length of Vocab :',len(Vocab))
print('Vocab :',Vocab)

Length of Vocab : 27
Vocab : ['l', 'o', 'w', '_', 'e', 's', 't', 'n', 'r', 'i', 'd', 'er', 'er_', 'ne', 'new', 'lo', 'low', 'newer_', 'low_', 'wi', 'wid', 'wider_', 'lowe', 'lowes', 'lowest', 'lowest_', 'new_']
